# 统计rtdetr的Patch和Cell粒度统计量

In [3]:
import os, glob
from concurrent.futures import ThreadPoolExecutor
import torch
from tqdm import tqdm


['000100042.pt', '002907322.pt', '002907324.pt', '002907329.pt', '002907330.pt', '002907332.pt', '002907333.pt', '002907336.pt', '002907342.pt', '002907344.pt']


In [25]:
def count_info(wsi_names, cell_nums):
    patch_nums = [len(cell_num) for cell_num in cell_nums]
    for i, patch_cell_num in enumerate(cell_nums):
        try:
            if max(patch_cell_num) > 300:
                print('[ERROR]', i, wsi_names[i], max(patch_cell_num), min(patch_cell_num), sum(patch_cell_num) / len(patch_cell_num))
        except Exception as e:
            print('[ERROR]', wsi_names[i], e)
    
    all_cell_nums = []
    for patch_cell_num in cell_nums:
        all_cell_nums.extend(patch_cell_num)

    print(f'patch_num max:{max(patch_nums)}, min:{min(patch_nums)}, mean:{sum(patch_nums) / len(patch_nums)}')
    print(f'len(all_cell_nums):{len(all_cell_nums)}')
    print(f'cell_num max:{max(all_cell_nums)}, min:{min(all_cell_nums)}, mean:{sum(all_cell_nums) / len(all_cell_nums)}')

In [27]:
def count_cell(feat_path):
    wsi_feat = torch.load(feat_path)
    cell_num = [patch_feat.shape[0] for patch_feat in wsi_feat]
    return cell_num

# 对比两个版本的rtdetr特征
root_path = '/data/wsi/TCTGC10k-features/rtdetr-v2-2025.1.6'
wsi_names = os.listdir(root_path)
wsi_names = sorted(wsi_names)

root_path2 = '/data/wsi/TCTGC10k-features/rtdetr-v3-2025.1.7'
wsi_names2 = os.listdir(root_path2)
wsi_names2 = sorted(wsi_names2)

wsi_paths = [os.path.join(root_path, wsi_name) for wsi_name in wsi_names]
wsi_paths2 = [os.path.join(root_path2, wsi_name) for wsi_name in wsi_names2]

# with ThreadPoolExecutor(max_workers=32) as executor:
#     cell_nums = list(tqdm(executor.map(count_cell, wsi_paths), total=len(wsi_paths)))
# count_info(wsi_names, cell_nums)

with ThreadPoolExecutor(max_workers=32) as executor:
    cell_nums2 = list(tqdm(executor.map(count_cell, wsi_paths2), total=len(wsi_paths2)))
count_info(wsi_names2, cell_nums2)



[ERROR] 220 005201503-T.pt 325 175 276.0
[ERROR] 226 005201537-N.pt 354 100 290.1818181818182
[ERROR] 227 005201542-N.pt 377 116 306.625
[ERROR] 228 005201543-N.pt 357 123 290.64516129032256
[ERROR] 229 005201544-N.pt 353 126 290.3333333333333
[ERROR] 230 005201549-BV.pt 397 82 291.59375
[ERROR] 231 005201558-BV.pt 307 19 219.61764705882354
[ERROR] 232 005201561-BV.pt 309 30 253.6451612903226
[ERROR] 241 006233133.pt 308 40 223.80555555555554
[ERROR] 260 006234085.pt 346 45 266.97222222222223
[ERROR] 261 006234215.pt 329 30 248.88888888888889
[ERROR] 262 006234218.pt 321 52 230.13888888888889
[ERROR] 263 006234221.pt 343 47 235.65714285714284
[ERROR] 264 006234225.pt 329 10 249.05555555555554
[ERROR] 265 006234240.pt 337 53 262.77777777777777
[ERROR] 267 006234252.pt 337 66 261.74285714285713
[ERROR] 268 006234253.pt 327 90 263.45714285714286
[ERROR] 278 006234426.pt 318 97 248.72222222222223
[ERROR] 316 006238293.pt 301 61 230.0
[ERROR] 327 006238372.pt 343 218 294.258064516129
[ERROR

100%|██████████| 9854/9854 [27:57<00:00,  5.88it/s]  


patch_num max:1666, min:20, mean:408.4723969961437
len(all_cell_nums):4025087
cell_num max:54, min:1, mean:4.645017610799468
